# Prevención Fuga de Clientes
El enfoque de la prevención es disminuir el impacto latente de la posibilidad de que alguno de los clientes deje de comprar productos. 
Con este fin se realiza la estrategia en 3 sentidos: <br>
**1.Motor de Recomendación** <br>
**Objetivo:** Aumentar la penetración de los productos a los clientes con el fin de fortalecer la relación comercial y su fidelización. <br>
Este enfoque hace parte de la primera parte del notebook, con el objetivo de generar un motor de recomendación con el cual se facilite la venta de nuevos productos por medio de estrategia de datos. <br>
*Notebook 1: Motor de recomendación* <br>

**2.Nueva segmentación de información con base en los datos de los clientes y sus transacciones** <br>
**Objetivo:** Generar nueva información de los clientes, con el objetivo de realizar una mejor gestión de estos y plantear estrategias con base en esta nueva característica de información desarrollada. <br>
El segundo enfoque es a través de una nueva segmentación, que permita de igual forma identificar no solo como y que es posible comprar, sino generar nuevas dimensiones de análisis, para que los equipos de business intelligence, comerciales y financieros, tengan más herramientas durante sus estrategias de negocio. <br>
*Notebook 2: Segmentación de datos* <br>

**3.Predecir la Fuga de clientes** <br>
**Objetivo:** Desarrollar un marcaje de fuga de clientes con base en la historia transaccional de cliente, permitiendo esto entrenar un modelo de Machine Learning para predecir la posible baja de clientes. <br>
El Tercer enfoque tiene dos frentes de trabajo, el primero permite establecer a través de variables de tendencias de compra y montos, si el cliente se encuentra "vivo" o no. Con base en este resultado, se activa el segundo frente, el cual a través de algoritmos de Machine Learning permite predecir la fuga de clientes con base en el marcaje realizado en la primera etapa. <br>
*Notebook 3: Predicción Fuga Clientes* <br>


### Fuente de datos
Los datos usando durante la tesis son de dominio publico y hacen referencia a un comercio de Brasil. <br>
Estos datos han sido modelados en un modelo dimensional con el fin de mejorar el performance del análisis del actual tesis y permitir generar modelos de reporting/dashboard con herramientas de BI.<br>
Los datos han sido migrados a la plataforma github con el fin que sean de dominio publico. <br>


##1. Motor de Recomendación

Durante esta etapa se realizarán dos enfoques de recomendación, el primero con base en productos y el segundo con base en el comportamiento de clientes.


### 1.1 Carga de librerias y datos

In [0]:
!pip install turicreate
# librerias requeridas
import turicreate as tc
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
%matplotlib inline
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score
import time
from sklearn.model_selection import train_test_split

In [3]:
## Localización y carga de los datos
url = 'https://github.com/masdatascience/TFM-AI/blob/master/data_model_completo.xlsx?raw=true'
datos_modelo = pd.ExcelFile(url)
datos_facturacion = pd.read_excel(datos_modelo, sheet_name='fac_txn')
datos_producto = pd.read_excel(datos_modelo, sheet_name='product')
datos_clientes = pd.read_excel(datos_modelo, sheet_name='customer')

# Se elimian los datos nulos de las dimensiones importantes del analisis que son productos y clientes
datos_facturacion = datos_facturacion.dropna(subset=['customer_id'])
datos_facturacion = datos_facturacion.dropna(subset=['product_id'])
# Se elimina posibles chargeback que se presenten
datos_facturacion = datos_facturacion.loc[datos_facturacion['total_value'] > 0]

print("Facturación:"+ str(datos_facturacion.shape) + " Clientes:"+str(datos_clientes.shape)+" Productos:"+str(datos_producto.shape))

Facturación:(112650, 21) Clientes:(96352, 4) Productos:(32951, 10)


### 1.2 Preparación de los datos 
En esta sección se estarán preparando tres enfoques de matrices con el fin de permitir desarrollar los motores de recomendación. La primera es la relación entre el cliente y los productos, la siguiente es una matriz normalizada y la última integra valores dummies para la comparación . 

#### 1.2.1 Relación Cliente - Producto



In [0]:
### Eliminar
data= datos_facturacion.copy()
datos_facturacion = datos_facturacion.head(10000)

In [39]:
matrix_usuario_producto =pd.crosstab(datos_facturacion['customer_id'],datos_facturacion['product_id'],values=datos_facturacion['total_value'], aggfunc='sum', margins=True, margins_name="total_value")
print("Matriz Cliente-Producto:" +str(matrix_usuario_producto.shape))

Matriz Cliente-Producto:(8776, 6089)


#### 1.2.2 Relación Cliente Producto - Normalizada

In [40]:
matrix_usuario_producto_norm = (matrix_usuario_producto-matrix_usuario_producto.min())/(matrix_usuario_producto.max()-matrix_usuario_producto.min())
# Se crea una tabla como resultado de la normalización  
d = matrix_usuario_producto_norm.reset_index() 
d.index.names = ['value_freq'] 
data_norm = pd.melt(d, id_vars=['customer_id'], value_name='value_freq').dropna()
print("Matriz normalizada:" +str(data_norm.shape))

Matriz normalizada:(14312, 3)


#### 1.2.3 Relación Cliente Producto - Normalizada

In [41]:
datos_dummy = datos_facturacion.copy()
datos_dummy['total_dummy'] = 1
print("Matriz Dummy:"+ str(datos_dummy.shape))

Matriz Dummy:(10000, 22)


#### 1.2.4 Se generan datos de entrenamiento y pruebas

In [0]:
train_data, test_data  = train_test_split(datos_facturacion, test_size=0.3 ,random_state=1)
train_data_dummy, test_data_dummy = train_test_split(datos_dummy, test_size=0.3 ,random_state=1)
train_data_norm, test_data_norm = train_test_split(data_norm, test_size=0.3 ,random_state=1)

#### 1.2.5  Se define función para ejecutar los tipos de motores de recomendación

In [0]:
# Se define una función que retorne un modelo resultado segun el enfoque que se requiere:
# 1. Popularity : Es el item que más se a consumido en todos los clientes
# 2. cosine :
# 3. Pearson: 
def modelo_requerido(modelo,datos ,campo ):

    train_data=tc.SFrame(datos)
    print("Modelo solicitado:"+ str(modelo))
    if modelo == 'producto':
        return  tc.popularity_recommender.create(train_data,user_id='customer_id', item_id='product_id', target=campo)
    elif modelo == 'distancia':
        return tc.item_similarity_recommender.create(train_data,user_id='customer_id', item_id='product_id', target=campo,similarity_type='cosine')
    elif modelo == 'pearson':
        return tc.item_similarity_recommender.create(train_data,user_id='customer_id', item_id='product_id', target=campo,similarity_type='pearson')
    else : 
        print("Error: Modelo solicitado no existe")

def productos_recomendado(modelo, usuarios, cantidad_recomendaciones):
     modelo.recommend(users=list(usuarios), k=cantidad_recomendaciones).print_rows(cantidad_recomendaciones)

In [0]:
# Se generan las recomendaciones por dataset
fac_producto = modelo_requerido('producto',train_data, 'total_value')
nor_producto = modelo_requerido('producto',train_data_norm,'value_freq' )
dum_producto = modelo_requerido('producto' ,train_data_dummy, 'total_dummy')

In [96]:
train_data_dummy.head()

,order_id,order_item_id,product_id,customer_id,customer_zip_code_prefix,shipping_limit_date,price,freight_value,total_value,credit_card,debit_card,boleto,voucher,not_defined,num_txn,order_status,order_purchase_timestamp,order_approved_at,order_delivered_carrier_date,order_delivered_customer_date,order_estimated_delivery_date,total_dummy
2228,0512bcd90b810913b8177d5324c5ee0d,1,68bf2e76db1d8e846042e54da87399ca,e4bb330e4e0967e8cdd333d1e8b1df2f,6700,2017-06-30 16:45:18,6990,1774,8764,8764.0,0.0,0.0,0.0,0.0,1.0,delivered,2017-06-22 16:28:38,2017-06-22 16:45:18,2017-06-23 10:57:29,2017-07-01 11:23:36,2017-07-18,1
5910,0d6350e1fa2acb1318e3a53d52333e25,1,4520766ec412348b8d4caa5e8a18c464,c4c85e60adccaf6b8ccb7ff2701f2e36,36572,2018-07-23 22:15:14,2749,1829,4578,4578.0,0.0,0.0,0.0,0.0,1.0,delivered,2018-07-19 22:00:14,2018-07-19 22:15:14,2018-07-20 09:28:00,2018-07-25 12:05:18,2018-08-06,1
1950,046cb513a5ed6d4f7506af8f88636609,1,68ad45d48d69404aeb71ce87e1b2c948,88613309c9c1df2902d6778360d5f1d9,5271,2018-06-27 23:31:52,5989,917,6906,6906.0,0.0,0.0,0.0,0.0,1.0,delivered,2018-06-24 23:13:35,2018-06-24 23:34:10,2018-06-25 14:44:00,2018-06-28 18:42:02,2018-07-11,1
2119,04cc9ab9d21b11d7aca691cf7facaaa1,1,7c55ea4aea1acf1ce11440010f5aa298,0e201fc3c7b9b02edb2d1a6e7f188912,3047,2018-05-09 04:15:07,39900,246,40146,0.0,0.0,53609.0,0.0,0.0,1.0,delivered,2018-04-30 09:13:44,2018-05-03 04:15:07,2018-05-03 17:20:00,2018-05-04 12:28:57,2018-05-14,1
5947,0d79a28c125a4b427a4ee97d86e4546b,1,aadff88486740e0b0ebe2be6c09476ae,fdf983831b2751f645ed24f62f70be6a,8665,2018-05-14 14:14:56,2990,771,3761,3761.0,0.0,0.0,0.0,0.0,1.0,delivered,2018-05-09 13:50:08,2018-05-09 14:14:56,2018-05-10 12:38:00,2018-05-12 14:48:39,2018-05-18,1


recommendations finished on 1000/96352 queries. users per second: 72207.4

recommendations finished on 2000/96352 queries. users per second: 68460.3

recommendations finished on 3000/96352 queries. users per second: 68184.9

recommendations finished on 4000/96352 queries. users per second: 68232.6

recommendations finished on 5000/96352 queries. users per second: 68230.5

...

recommendations finished on 93000/96352 queries. users per second: 66919.6

recommendations finished on 94000/96352 queries. users per second: 66853.1

recommendations finished on 95000/96352 queries. users per second: 66535.6

recommendations finished on 96000/96352 queries. users per second: 65970.2

+-------------------------------+-------------------------------+----------+------+
|          customer_id          |           product_id          |  score   | rank |
+-------------------------------+-------------------------------+----------+------+
| 0000366f3b9a7992bf8c76cfdf... | 34f99d82cfc355d08d8db780d1... | 312650.0 |  1   |
| 0000366f3b9a7992bf8c76cfdf... | a233df9a388d27dbdfd31731d4... | 278416.0 |  2   |
| 0000366f3b9a7992bf8c76cfdf... | a3cd9517ebf5a50dca25acce54... | 271336.0 |  3   |
| 0000b849f77a49e4a4ce2b2a4c... | 34f99d82cfc355d08d8db780d1... | 312650.0 |  1   |
| 0000b849f77a49e4a4ce2b2a4c... | a233df9a388d27dbdfd31731d4... | 278416.0 |  2   |
| 0000b849f77a49e4a4ce2b2a4c... | a3cd9517ebf5a50dca25acce54... | 271336.0 |  3   |
+-------------------------------+-------------------------------+----------+------+
[289056 rows x 4 columns]



In [0]:
# se definen parametros para recomendar el nu´meruo de productos a recomendar y una previsualziación definida del resultado
user_id = 'customer_id'
item_id = 'product_id'
users_to_recommend = list(datos_clientes[user_id])
n_rec = 5 # number of items to recommend
n_display = 5 # to display the first few rows in an output dataset
# Se define una función que retorne un modelo resultado segun el enfoque que se requiere:
# 1. Popularity : Es el item que más se a consumido en todos los clientes
# 2. cosine :
# 3. Pearson: 
def model(train_data, name, user_id, item_id, target, users_to_recommend, n_rec, n_display):
    if name == 'popularity':
        model = tc.popularity_recommender.create(train_data, user_id=user_id, item_id=item_id, target=target);
    elif name == 'cosine':
        model = tc.item_similarity_recommender.create(train_data, user_id=user_id, item_id=item_id, target=target, similarity_type='cosine')
    elif name == 'pearson':
        model = tc.item_similarity_recommender.create(train_data,user_id=user_id,item_id=item_id,target=target,similarity_type='pearson')
        
    recom = model.recommend(users=users_to_recommend, k=3);
    recom.print_rows(6)
    return model

In [0]:
name = 'popularity'
target = 'total_value'
train_dataF=tc.SFrame(train_data_dummy)
test_dataF=tc.SFrame(test_data_dummy)
popularity_model = model(train_dataF, name, user_id, item_id, target, users_to_recommend, n_rec, n_display)

### 1.3 Motor de recomendación por productos
Se realiza la ejecución del modelo desde los tres datasets definidos de información, con el fin de asegurar el mejor valor a ser entregado al equipo de ventas y analistas de la organización

In [0]:
name = 'popularity'
target = 'total_value'
train_dataF=tc.SFrame(train_data)
test_dataF=tc.SFrame(test_data)
popularity_model = model(train_dataF, name, user_id, item_id, target, users_to_recommend, n_rec, n_display)

In [0]:
pd.crosstab(datos_facturacion['customer_id'],datos_facturacion['product_id'],values=datos_facturacion['total_value'], aggfunc='sum')  # margins=True, margins_name="total_value")


### 1. Motor de recomendación

In [0]:
datos_facturacion.head()
cols = ['customer_id', 'product_id','total_value']
datos_proceso= pd.DataFrame(datos_facturacion, columns=cols, dtype=object)
datos_proceso.dtypes
datos_proceso.head()

,customer_id,product_id,total_value
0,871766c5855e863f6eccc05f988b23cb,4244733e06e7ecb4970a6e2683c13e61,7219
1,eb28e67c4c0b83846050ddfb8a35d051,e5f2d52b802189ee658865ca93d83a8f,25983
2,3818d81c6709e39d06b2738a8d3a2474,c777355d18b72b67abbeef9df44fd0fd,21687
3,af861d436cfc08b2c2ddefd0ba074622,7634da152a4610f1595efa32f14722fc,2578
4,64b576fb70d441e8f1b2d7d446e483c5,ac6c3623068f30de03045865e4e10089,21804


In [0]:
datos_dummy = datos_facturacion.copy()
datos_dummy['total_dummy'] = 1
datos_dummy.head()

,order_id,order_item_id,product_id,customer_id,customer_zip_code_prefix,seller_id,shipping_limit_date,price,freight_value,total_value,...,voucher,not_defined,num_txn,order_status,order_purchase_timestamp,order_approved_at,order_delivered_carrier_date,order_delivered_customer_date,order_estimated_delivery_date,total_dummy
0,00010242fe8c5a6d1ba2dd792cb16214,1,4244733e06e7ecb4970a6e2683c13e61,871766c5855e863f6eccc05f988b23cb,28013,48436dade18ac8b2bce089ec2a041202,2017-09-19 09:45:35,5890,1329,7219,...,0,0,1,delivered,2017-09-13 08:59:02,2017-09-13 09:45:35,2017-09-19 18:34:16,2017-09-20 23:43:48,2017-09-29,1
1,00018f77f2f0320c557190d7a144bdd3,1,e5f2d52b802189ee658865ca93d83a8f,eb28e67c4c0b83846050ddfb8a35d051,15775,dd7ddc04e1b6c2c614352b383efe2d36,2017-05-03 11:05:13,23990,1993,25983,...,0,0,1,delivered,2017-04-26 10:53:06,2017-04-26 11:05:13,2017-05-04 14:35:00,2017-05-12 16:04:24,2017-05-15,1
2,000229ec398224ef6ca0657da4fc703e,1,c777355d18b72b67abbeef9df44fd0fd,3818d81c6709e39d06b2738a8d3a2474,35661,5b51032eddd242adc84c38acab88f23d,2018-01-18 14:48:30,19900,1787,21687,...,0,0,1,delivered,2018-01-14 14:33:31,2018-01-14 14:48:30,2018-01-16 12:36:48,2018-01-22 13:19:16,2018-02-05,1
3,00024acbcdf0a6daa1e931b038114c75,1,7634da152a4610f1595efa32f14722fc,af861d436cfc08b2c2ddefd0ba074622,12952,9d7a1d34a5052409006425275ba1c2b4,2018-08-15 10:10:18,1299,1279,2578,...,0,0,1,delivered,2018-08-08 10:00:35,2018-08-08 10:10:18,2018-08-10 13:28:00,2018-08-14 13:32:39,2018-08-20,1
4,00042b26cf59d7ce69dfabb4e55b4fd9,1,ac6c3623068f30de03045865e4e10089,64b576fb70d441e8f1b2d7d446e483c5,13226,df560393f3a51e74553ab94004ba5c87,2017-02-13 13:57:51,19990,1814,21804,...,0,0,1,delivered,2017-02-04 13:57:51,2017-02-04 14:10:13,2017-02-16 09:46:09,2017-03-01 16:42:31,2017-03-17,1


### 1.1 Colaborativo

In [0]:
# Se genera una pivot especifica para los campos requeridos del motor de recomendación 
matrix_usuario_producto = datos_proceso.pivot_table(index='customer_id',columns='product_id', values='total_value',aggfunc='sum')
# Se comprueba resultado
matrix_usuario_producto.head()

Se procede a normalizar la compras para ser comparables entre los valores

In [0]:
matrix_usuario_producto_norm = (matrix_usuario_producto-matrix_usuario_producto.min())/(matrix_usuario_producto.max()-matrix_usuario_producto.min())
# Se crea una tabla como resultado de la normalización  
d = matrix_usuario_producto_norm.reset_index() 
d.index.names = ['value_freq'] 
data_norm = pd.melt(d, id_vars=['customer_id'], value_name='value_freq').dropna()
print(data_norm.shape)
data_norm.head()

TypeError: Could not operate array([24210.,  4909.,  9290., ...,  6811.,  4104., 10012.]) with block values 

In [0]:
def split_data(data):
    '''
    Splits dataset into training and test set.
    
    Args:
        data (pandas.DataFrame)
        
    Returns
        train_data (tc.SFrame)
        test_data (tc.SFrame)
    '''
    train, test = train_test_split(data, test_size = .2)
    train_data = tc.SFrame(train)
    test_data = tc.SFrame(test)
    return train_data, test_data
  
  

#train_data, test_data = train_test_split(datos_facturacion, test_size = .2)
#train_data, test_data = train_test_split(datos_dummy, test_size = .2)


train_data, test_data = split_data(datos_facturacion)
train_data_dummy, test_data_dummy = split_data(datos_dummy)
train_data_norm, test_data_norm = split_data(data_norm)

In [0]:
#A more complicated but common approach to predict purchase items is collaborative filtering. 
#I will discuss more about the popularity model and collaborative filtering in the later section. For now, let’s first define our variables to use in the models:

# constant variables to define field names include:
user_id = 'customer_id'
item_id = 'product_id'
users_to_recommend = list(datos_clientes[user_id])
n_rec = 10 # number of items to recommend
n_display = 30 # to display the first few rows in an output dataset

In [0]:
def model(train_data, name, user_id, item_id, target, users_to_recommend, n_rec, n_display):
    if name == 'popularity':
        model = tc.popularity_recommender.create(train_data, 
                                                    user_id=user_id, 
                                                    item_id=item_id, 
                                                    target=target)
    elif name == 'cosine':
        model = tc.item_similarity_recommender.create(train_data, 
                                                    user_id=user_id, 
                                                    item_id=item_id, 
                                                    target=target, 
                                                    similarity_type='cosine')
    elif name == 'pearson':
        model = tc.item_similarity_recommender.create(train_data, 
                                                    user_id=user_id, 
                                                    item_id=item_id, 
                                                    target=target, 
                                                    similarity_type='pearson')
        
    recom = model.recommend(users=users_to_recommend, k=n_rec)
    recom.print_rows(n_display)
    return model

In [0]:
name = 'popularity'
target = 'total_value'
popularity_model = model(train_data, name, user_id, item_id, target, users_to_recommend, n_rec, n_display)

In [0]:
name = 'popularity'
target = 'total_dummy'
pop_dummy = model(train_data_dummy, name, user_id, item_id, target, users_to_recommend, n_rec, n_display)

In [0]:
name = 'popularity'
target = 'value_freq'
pop_norm = model(train_data_norm, name, user_id, item_id, target, users_to_recommend, n_rec, n_display)

In [0]:
#colaborativa
name = 'cosine'
target = 'total_value'
cos = model(train_data, name, user_id, item_id, target, users_to_recommend, n_rec, n_display)

In [0]:
name = 'cosine'
target = 'total_dummy'
cos_dummy = model(train_data_dummy, name, user_id, item_id, target, users_to_recommend, n_rec, n_display)


In [0]:

name = 'cosine'
target = 'value_freq'
cos_norm = model(train_data_norm, name, user_id, item_id, target, users_to_recommend, n_rec, n_display)


In [0]:
name = 'pearson'
target = 'total_value'
pear = model(train_data, name, user_id, item_id, target, users_to_recommend, n_rec, n_display)

In [0]:
name = 'pearson'
target = 'total_dummy'
pear_dummy = model(train_data_dummy, name, user_id, item_id, target, users_to_recommend, n_rec, n_display)

In [0]:
name = 'pearson'
target = 'value_freq'
pear_norm = model(train_data_norm, name, user_id, item_id, target, users_to_recommend, n_rec, n_display)


In [0]:
models_w_counts = [popularity_model, cos, pear]
models_w_dummy = [pop_dummy, cos_dummy, pear_dummy]
models_w_norm = [pop_norm, cos_norm, pear_norm]
names_w_counts = ['Popularity Model on Purchase Counts', 'Cosine Similarity on Purchase Counts', 'Pearson Similarity on Purchase Counts']
names_w_dummy = ['Popularity Model on Purchase Dummy', 'Cosine Similarity on Purchase Dummy', 'Pearson Similarity on Purchase Dummy']
names_w_norm = ['Popularity Model on Scaled Purchase Counts', 'Cosine Similarity on Scaled Purchase Counts', 'Pearson Similarity on Scaled Purchase Counts']

In [0]:
eval_counts = tc.recommender.util.compare_models(test_data, models_w_counts, model_names=names_w_counts)

In [0]:
eval_dummy = tc.recommender.util.compare_models(test_data_dummy, models_w_dummy, model_names=names_w_dummy)


In [0]:
eval_norm = tc.recommender.util.compare_models(test_data_norm, models_w_norm, model_names=names_w_norm)

In [0]:
final_model = tc.item_similarity_recommender.create(tc.SFrame(data_norm), 
                                            user_id=user_id, 
                                            item_id=item_id, 
                                            target='value_freq', similarity_type='cosine')
recom = final_model.recommend(users=users_to_recommend, k=n_rec)
recom.print_rows(n_display)

In [0]:
df_rec = recom.to_dataframe()
print(df_rec.shape)
df_rec.head()

In [0]:
def create_output(model, users_to_recommend, n_rec, print_csv=True):
    recomendation = model.recommend(users=users_to_recommend, k=n_rec)
    df_rec = recomendation.to_dataframe()
    df_rec['recommendedProducts'] = df_rec.groupby([user_id])[item_id] \
        .transform(lambda x: '|'.join(x.astype(str)))
    df_output = df_rec[['customer_id', 'recommendedProducts']].drop_duplicates() \
        .sort_values('customer_id').set_index('customer_id')
    if print_csv:
        df_output.to_csv(directorio+'option1_recommendation.csv')
        print("An output file can be found in 'output' folder with name 'option1_recommendation.csv'")
    return df_output

In [0]:
df_output = create_output(pear_norm, users_to_recommend, n_rec, print_csv=True)
print(df_output.shape)
df_output.head()

In [0]:
def customer_recomendation(customer_id):
    if customer_id not in df_output.index:
        print('Customer not found.')
        return customer_id
    return df_output.loc[customer_id]

In [0]:
customer_recomendation('000379cdec625522490c315e70c7a9fb')

In [0]:
#train_data.groupby('product_id').agg({'total_value':'mean'}).sort_values(ascending=False)

In [0]:
# se genera el enfoque de recomendación con base colaborativo (usuarios a usuarios) cuando tienen historia
matrix_usuarios = pd.DataFrame(cosine_similarity(matrix_usuario_producto.fillna(0)))
# Se verifica resultado
matrix_usuarios.head()

In [0]:
# se cambian los nombres de las columnas con los IDs de los usuarios
matrix_usuarios.columns = matrix_usuario_producto.index
matrix_usuarios['c_id'] = matrix_usuario_producto.index
matrix_usuarios = matrix_usuarios.set_index('c_id')
matrix_usuarios.head(10)

#### Aplicación para un cliente
En este momento ya se tiene la nueva herramienta de recomendación para ser entregada a los vendedores, a traves de esta se podra tomar a un cliente especifico y generarle una estrategia personalizada de penetración con base en la información historica de clientes similares. <br>
Se visualizará a continuación como es el resultado de analizar un cliente con esta herramienta. El objtivo de este producto es que sea integrado a una herramienta BI , la cual facilite el uso de la misma a traves de dispositivos mobiles brindandole así al asesor la posibilidad de ir directamente a visitar a los clientes como parte de esta estrategia de fidelización del cliente. <br>
Para este ejemplo se tomará al primer cliente con el id : **0281f03348b06722b7e0f92455daafd4** , el cual se visualiza en la anterior matriz. 


In [0]:
# Se identifican los  usuarios que se relacionan mas con el un cliente analizado, apoyando realizar una penentración de productos proactiva
matrix_usuarios.loc['0005ef4cd20d2893f0d9fbd94d3c0d97'].sort_values(ascending=False).head(10)  # se obtienen los primeros cuatro clientes
# Ahora se analizará como se pueden penetrarle nuevos productos al cliente 17396 con base en la venta de 12347

In [0]:
# Se identifican los productos comprados por aeb3d464f6b83eecd79d6fd58d6208fb    
productos_comprados_ref = set(matrix_usuarios.loc['0005ef4cd20d2893f0d9fbd94d3c0d97'].iloc[matrix_usuarios.loc['0005ef4cd20d2893f0d9fbd94d3c0d97'].to_numpy().nonzero()].index)
print('Productos comprados cliente referencia:' + str(productos_comprados_ref))
# Se identifican los productos comprados por d482c96cc23e02b643f79768912442c2        
productos_comprados_target = set(matrix_usuarios.loc['bf429f20c75a39eab83442cda8e59832'].iloc[matrix_usuarios.loc['bf429f20c75a39eab83442cda8e59832'].to_numpy().nonzero()].index)
print('Productos cliente a venderle nuevo productos:' + str(productos_comprados_target))
# Producto con oportunidad de penetrar al cliente d482c96cc23e02b643f79768912442c2
recomendados = productos_comprados_ref - productos_comprados_target
print('Productos a venderle:' + str(recomendados))



In [0]:
datos_producto.loc[datos_producto['product_id'].isin(recomendados),['product_id', 'product_category_name_english']].drop_duplicates().set_index('product_id')

### 1.2 Producto

In [0]:
# se genera el enfoque de recomendación con base en los productos (producto a producto), lo cual es un factor interesante para los nuevos clientes que no tienen historia o
# sobre clientes que estan en el proceso de lead a new business. Para este enfoque se realiza a traves de la matriz transpuesta original ya que su enfoque es con vista al producto. 
matrix_producto = pd.DataFrame(cosine_similarity(matrix_usuario_producto.fillna(0).T))
# se cambian los nombres de las columnas con los IDs de los productos
matrix_producto.columns = matrix_usuario_producto.T.index
matrix_producto['p_id'] = matrix_usuario_producto.T.index
matrix_producto = matrix_producto.set_index('p_id')
# Se verifica resultado
matrix_producto.head()

In [0]:
# Se identifican el top de clientes
top_10_prod = list(matrix_producto.loc['0009406fd7479715e4bef61dd91f2462'].sort_values(ascending=False).iloc[:10].index)
datos_producto.loc[datos_producto['product_id'].isin(top_10_prod)]

## 2. Segmentación de Clientes

In [0]:
from numpy import inf
def prevDivision(numerador, denominador):
  return numerador / denominador if denominador else 0
  

In [0]:
# Agrupo la información de cliente para poder segmentarlo 

informacion_cliente = datos_facturacion.groupby('customer_id').agg({
    'total_value': sum,
    'product_id': lambda x: x.nunique(),
    'num_txn': sum,
    
}).fillna(0)

informacion_cliente.columns = ['IMP_TOTAL_VENTAS', 'NUM_TOTAL_PRODUCTOS','NUM_TOTAL_TXN']
# ESTE HAY QUE CAMBIARLO POR LA MEDIANA
informacion_cliente['IMP_VENTA_PROMEDIO'] = informacion_cliente['IMP_TOTAL_VENTAS']/informacion_cliente['NUM_TOTAL_TXN']
informacion_cliente.head()

In [0]:
# Limpiar posibles errores
informacion_cliente=informacion_cliente.dropna()
informacion_cliente=informacion_cliente[~informacion_cliente.isin([np.nan, np.inf, -np.inf]).any(1)]

# se pinta el codo
X = np.array(informacion_cliente[["IMP_TOTAL_VENTAS","NUM_TOTAL_PRODUCTOS","NUM_TOTAL_TXN","IMP_VENTA_PROMEDIO"]].dropna())
Nc = range(1, 20)
kmeans = [KMeans(n_clusters=i) for i in Nc]
kmeans
score = [kmeans[i].fit(X).score(X) for i in range(len(kmeans))]
score

plt.plot(Nc,score, 'ro-', markersize=8, lw=2)
plt.xlabel('Number of Clusters')
plt.ylabel('Score')
plt.title('Elbow Curve')
plt.show()

In [0]:
# Feature Scaling
from sklearn.preprocessing import StandardScaler

X = informacion_cliente.iloc[:, :].values
sc_X = StandardScaler()
X= sc_X.fit_transform(X)
wcss = [] #With in cluster sum of squers(Inertia)
#n_clusters is no.of clusters given by this method,
#k-means++ is an random initialization methods for centriods to avoid random
#intialization trap,
#max_iter is max no of iterations defined when k-means is running
#n_init is no of times k-means will run with different initial centroids

for i in range(1,11): #From 2-10 doing multiple random initializations can make a huge difference to find a better local optima
    kmeans = KMeans(n_clusters = i, init ='k-means++',max_iter=300,n_init=10)
    kmeans.fit(X)
    wcss.append(kmeans.inertia_)
plt.plot(range(1,11) , wcss)
plt.title('The Elbow Method')
plt.xlabel('Number Of Customer Clusters(customer type clusters)')
plt.ylabel('With in cluster sum of squers(WCSS)')
plt.show()

In [0]:
# Fitting K-Means to the dataset
kmeans = KMeans(n_clusters = 3, init = 'k-means++')
y_kmeans = kmeans.fit_predict(X)

# Visualising the clusters
plt.scatter(X[y_kmeans == 0, 0], X[y_kmeans == 0, 1], s = 50, c = 'red', label = 'Customer Type 1')
plt.scatter(X[y_kmeans == 1, 0], X[y_kmeans == 1, 1], s = 50, c = 'blue', label = 'Customer Type 2')
plt.scatter(X[y_kmeans == 2, 0], X[y_kmeans == 2, 1], s = 50, c = 'green', label = 'Customer Type 3')
plt.scatter(kmeans.cluster_centers_[:, 0], kmeans.cluster_centers_[:, 1], s = 100, c = 'yellow', label = 'Centroids')
plt.title('Type Of Customers(customer type clusters)')
plt.xlabel('Number of items Purchased(Quantity)')
plt.ylabel('Product price per unit in sterling(Unit Price)')
plt.legend()
plt.show()

x=[];y=[]
for i in range(4339):
    x.append(X[i][0])
    y.append(X[i][1])
plt.scatter(x,y)
plt.title('Plot of training data')
plt.xlabel('Number of items Purchased(Quantity)')
plt.ylabel('Product price per unit in sterling(Unit Price)')
plt.show()


In [0]:
num_clusters= 4

#X = np.array(informacion_cliente[["IMP_TOTAL_VENTAS","NUM_TOTAL_PRODUCTOS","NUM_TOTAL_TXN","IMP_VENTA_PROMEDIO"]])

# Normalizo los datos previo a la segmentación
ranking_cliente = informacion_cliente.rank(method='first')
cliente_normalizado = (ranking_cliente - ranking_cliente.mean()) / ranking_cliente.std()
kmeans = KMeans(n_clusters=num_clusters).fit(cliente_normalizado[['IMP_TOTAL_VENTAS', 'NUM_TOTAL_TXN', 'IMP_VENTA_PROMEDIO']])


cliente_clusters = cliente_normalizado[['IMP_TOTAL_VENTAS', 'NUM_TOTAL_TXN', 'IMP_VENTA_PROMEDIO']].copy(deep=True)
cliente_clusters['Cluster'] = kmeans.labels_
cliente_clusters.groupby('Cluster').count()['IMP_TOTAL_VENTAS']



In [0]:
# clustering ventas vs cantidad
plt.scatter(
    cliente_clusters.loc[cliente_clusters['Cluster'] == 0]['NUM_TOTAL_TXN'], 
    cliente_clusters.loc[cliente_clusters['Cluster'] == 0]['IMP_TOTAL_VENTAS'],
    c='blue'
)

plt.scatter(
    cliente_clusters.loc[cliente_clusters['Cluster'] == 1]['NUM_TOTAL_TXN'], 
    cliente_clusters.loc[cliente_clusters['Cluster'] == 1]['IMP_TOTAL_VENTAS'],
    c='red'
)

plt.scatter(
    cliente_clusters.loc[cliente_clusters['Cluster'] == 2]['NUM_TOTAL_TXN'], 
    cliente_clusters.loc[cliente_clusters['Cluster'] == 2]['IMP_TOTAL_VENTAS'],
    c='orange'
)

plt.scatter(
    cliente_clusters.loc[cliente_clusters['Cluster'] == 3]['NUM_TOTAL_TXN'], 
    cliente_clusters.loc[cliente_clusters['Cluster'] == 3]['IMP_TOTAL_VENTAS'],
    c='green'
)


plt.scatter(
    cliente_clusters.loc[cliente_clusters['Cluster'] == 4]['NUM_TOTAL_TXN'], 
    cliente_clusters.loc[cliente_clusters['Cluster'] == 4]['IMP_TOTAL_VENTAS'],
    c='yellow'
)


plt.scatter(
    cliente_clusters.loc[cliente_clusters['Cluster'] == 5]['NUM_TOTAL_TXN'], 
    cliente_clusters.loc[cliente_clusters['Cluster'] == 5]['IMP_TOTAL_VENTAS'],
    c='black'
)


plt.title('Ventas  vs. No. Transacciones Clusters')
plt.xlabel('Número Transacciones')
plt.ylabel('Total Ventas')

plt.grid()
plt.show()

# clustering cantidad vs promedio de facturación
plt.scatter(
    cliente_clusters.loc[cliente_clusters['Cluster'] == 0]['NUM_TOTAL_TXN'], 
    cliente_clusters.loc[cliente_clusters['Cluster'] == 0]['IMP_VENTA_PROMEDIO'],
    c='blue'
)

plt.scatter(
    cliente_clusters.loc[cliente_clusters['Cluster'] == 1]['NUM_TOTAL_TXN'], 
    cliente_clusters.loc[cliente_clusters['Cluster'] == 1]['IMP_VENTA_PROMEDIO'],
    c='red'
)

plt.scatter(
    cliente_clusters.loc[cliente_clusters['Cluster'] == 2]['NUM_TOTAL_TXN'], 
    cliente_clusters.loc[cliente_clusters['Cluster'] == 2]['IMP_VENTA_PROMEDIO'],
    c='orange'
)

plt.scatter(
    cliente_clusters.loc[cliente_clusters['Cluster'] == 3]['NUM_TOTAL_TXN'], 
    cliente_clusters.loc[cliente_clusters['Cluster'] == 3]['IMP_VENTA_PROMEDIO'],
    c='green'
)

plt.scatter(
    cliente_clusters.loc[cliente_clusters['Cluster'] == 4]['NUM_TOTAL_TXN'], 
    cliente_clusters.loc[cliente_clusters['Cluster'] == 4]['IMP_VENTA_PROMEDIO'],
    c='yellow'
)
plt.scatter(
    cliente_clusters.loc[cliente_clusters['Cluster'] == 5]['NUM_TOTAL_TXN'], 
    cliente_clusters.loc[cliente_clusters['Cluster'] == 5]['IMP_VENTA_PROMEDIO'],
    c='black'
)
plt.title('Promedio Ventas vs. No. Transacciones Clusters')
plt.xlabel('Número Transacciones')
plt.ylabel('Promedio Ventas')

plt.grid()
plt.show()
# clustering ventas vs promedio de facturación
plt.scatter(
    
    cliente_clusters.loc[cliente_clusters['Cluster'] == 0]['IMP_TOTAL_VENTAS'], 
    cliente_clusters.loc[cliente_clusters['Cluster'] == 0]['IMP_VENTA_PROMEDIO'],
    c='blue'
)

plt.scatter(
    cliente_clusters.loc[cliente_clusters['Cluster'] == 1]['IMP_TOTAL_VENTAS'], 
    cliente_clusters.loc[cliente_clusters['Cluster'] == 1]['IMP_VENTA_PROMEDIO'],
    c='red'
)

plt.scatter(
    cliente_clusters.loc[cliente_clusters['Cluster'] == 2]['IMP_TOTAL_VENTAS'], 
    cliente_clusters.loc[cliente_clusters['Cluster'] == 2]['IMP_VENTA_PROMEDIO'],
    c='orange'
)

plt.scatter(
    cliente_clusters.loc[cliente_clusters['Cluster'] == 3]['IMP_TOTAL_VENTAS'], 
    cliente_clusters.loc[cliente_clusters['Cluster'] == 3]['IMP_VENTA_PROMEDIO'],
    c='green'
)

plt.scatter(
    cliente_clusters.loc[cliente_clusters['Cluster'] == 4]['IMP_TOTAL_VENTAS'], 
    cliente_clusters.loc[cliente_clusters['Cluster'] == 4]['IMP_VENTA_PROMEDIO'],
    c='yellow'
)

plt.scatter(
    cliente_clusters.loc[cliente_clusters['Cluster'] == 5]['IMP_TOTAL_VENTAS'], 
    cliente_clusters.loc[cliente_clusters['Cluster'] == 5]['IMP_VENTA_PROMEDIO'],
    c='black'
)

plt.title('Total Ventas vs. Promedio Ventas Clusters')
plt.xlabel('Total Ventas')
plt.ylabel('Promedio Ventas')



In [0]:
# identificar el número de clientes por cluster
for n_cluster in [3,4,5,6,7,8]:
    kmeans = KMeans(n_clusters=n_cluster).fit(
        cliente_normalizado[['IMP_TOTAL_VENTAS', 'NUM_TOTAL_TXN', 'IMP_VENTA_PROMEDIO']]
    )
    silhouette_avg = silhouette_score(
        cliente_normalizado[['IMP_TOTAL_VENTAS', 'NUM_TOTAL_TXN', 'IMP_VENTA_PROMEDIO']], 
        kmeans.labels_
    )
    
    print('Scoring silueta para %i Clusters: %0.4f' % (n_cluster, silhouette_avg))

In [0]:
# identificar el número de clientes por cluster
for n_cluster in [3,4,5,6,7,8]:
    kmeans = KMeans(n_clusters=n_cluster).fit(
        cliente_normalizado[['IMP_TOTAL_VENTAS', 'NUM_TOTAL_TXN']]
    )
    silhouette_avg = silhouette_score(
        cliente_normalizado[['IMP_TOTAL_VENTAS', 'NUM_TOTAL_TXN']], 
        kmeans.labels_
    )
    
    print('Scoring silueta para %i Clusters: %0.4f' % (n_cluster, silhouette_avg))

In [0]:
# identificar el número de clientes por cluster
for n_cluster in [3,4,5,6,7,8]:
    kmeans = KMeans(n_clusters=n_cluster).fit(
        cliente_normalizado[['IMP_TOTAL_VENTAS', 'NUM_TOTAL_PRODUCTOS', 'IMP_VENTA_PROMEDIO']]
    )
    silhouette_avg = silhouette_score(
        cliente_normalizado[['IMP_TOTAL_VENTAS', 'NUM_TOTAL_PRODUCTOS', 'IMP_VENTA_PROMEDIO']], 
        kmeans.labels_
    )
    
    print('Scoring silueta para %i Clusters: %0.4f' % (n_cluster, silhouette_avg))

In [0]:
# identificar el número de clientes por cluster
for n_cluster in [3,4,5,6,7,8]:
    kmeans = KMeans(n_clusters=n_cluster).fit(
        cliente_normalizado[['IMP_TOTAL_VENTAS', 'NUM_TOTAL_PRODUCTOS', 'NUM_TOTAL_TXN']]
    )
    silhouette_avg = silhouette_score(
        cliente_normalizado[['IMP_TOTAL_VENTAS', 'NUM_TOTAL_PRODUCTOS', 'NUM_TOTAL_TXN']], 
        kmeans.labels_
    )
    
    print('Scoring silueta para %i Clusters: %0.4f' % (n_cluster, silhouette_avg))

In [0]:
# identificar el número de clientes por cluster
for n_cluster in [3,4,5,6,7,8]:
    kmeans = KMeans(n_clusters=n_cluster).fit(
        cliente_normalizado[['IMP_VENTA_PROMEDIO', 'NUM_TOTAL_PRODUCTOS', 'NUM_TOTAL_TXN']]
    )
    silhouette_avg = silhouette_score(
        cliente_normalizado[['IMP_VENTA_PROMEDIO', 'NUM_TOTAL_PRODUCTOS', 'NUM_TOTAL_TXN']], 
        kmeans.labels_
    )
    
    print('Scoring silueta para %i Clusters: %0.4f' % (n_cluster, silhouette_avg))

## Generando nueva data al dataset actual

In [0]:
kmeans = KMeans(n_clusters=4).fit(
    cliente_normalizado[['IMP_TOTAL_VENTAS', 'NUM_TOTAL_TXN']]
)

In [0]:
cliente_normalizado['DATA_SEG'] = kmeans.labels_

In [0]:
cliente_normalizado.head()
kmeans.cluster_centers_

In [0]:
high_value_cluster = cliente_normalizado.loc[cliente_normalizado['DATA_SEG'] == 3]
high_value_cluster.head()

In [0]:
informacion_cliente.loc[high_value_cluster.index].describe()

In [0]:
datos_facturacion.head()

In [0]:
pd.DataFrame(
    datos_facturacion.loc[
        datos_facturacion['customer_id'].isin(high_value_cluster.index)
    ].groupby('product_id').count()[
        'total_value'
    ].sort_values(ascending=False).head()
)

## 3. Matriz de Factorización

Una vez utilizado los posibles escenarios para las recomendaciónde productos, se procede a generar el motor de recomendación .  Para ello se realiza la matriz de Factorización, la cual se utilizo como referencia el sigueinte paper [Probabilistic Matrix Factorization](https://https://papers.nips.cc/paper/3208-probabilistic-matrix-factorization.pdf)

In [0]:
matrix_usuario_producto.values

**Enfoque Cliente**

In [0]:
from sklearn.neighbors import NearestNeighbors
from scipy.sparse import csr_matrix
# Se convierte la table pivote de cliente - producto a una sparse matrix
mat_cliente_prod = csr_matrix(matrix_usuario_producto.values)
# Se procede a identificar la relación entre los clientes
model_knn = NearestNeighbors(metric='cosine', algorithm='brute', n_neighbors=20, n_jobs=-1)


In [0]:
final_productos = matrix_usuario_producto.fillna(matrix_usuario_producto.mean(axis=0))
final_productos.head()

In [0]:
final_clientes = matrix_usuario_producto.apply(lambda row: row.fillna(0), axis=1)
final_clientes.head()

## Similaridad Usuarios

In [0]:
b = cosine_similarity(final_clientes)
np.fill_diagonal(b, 0 )
sim_usuario = pd.DataFrame(b,index=final_clientes.index)
sim_usuario.columns=final_clientes.index
sim_usuario.head()

In [0]:
# user similarity on replacing NAN by item(movie) avg
cosine = cosine_similarity(final_productos)
np.fill_diagonal(cosine, 0 )
sim_producto =pd.DataFrame(cosine,index=final_productos.index)
sim_producto.columns=final_clientes.index
sim_producto.head()

##Función para encontrar similares

In [0]:
def identificar_similares(data,cantidad):
    order = np.argsort(data.values, axis=1)[:, :cantidad]
    data = data.apply(lambda x: pd.Series(x.sort_values(ascending=False).iloc[:cantidad].index, 
          index=['top{}'.format(i) for i in range(1, cantidad+1)]), axis=1)
    return data

In [0]:
identificar_similares(sim_producto,20)

In [0]:
identificar_similares(sim_usuario,20)

## Función similitud de productos entre usuarios

In [0]:
def identificar_productos( cliente1, cliente2 ):
    common_movies = datos_facturacion[datos_facturacion.c_id == cliente1].merge(
    datos_facturacion[datos_facturacion.c_id == cliente2],
    on = "p_id",
    how = "inner" )
    return common_movies.merge( datos_producto, on = 'p_id' )

In [0]:
#identificar_productos(7,16)

#Identificar Churn

In [0]:
import seaborn as sns
import matplotlib.pyplot as plt
!pip install lifetimes
from lifetimes.utils import *
from lifetimes import BetaGeoFitter,GammaGammaFitter
from lifetimes.plotting import plot_probability_alive_matrix, plot_frequency_recency_matrix, plot_period_transactions, plot_cumulative_transactions,plot_incremental_transactions
from lifetimes.generate_data import beta_geometric_nbd_model
from lifetimes.plotting import plot_calibration_purchases_vs_holdout_purchases, plot_period_transactions,plot_history_alive

In [0]:
datos_modelo = pd.ExcelFile(directorio+'data_model.xlsx')
datos_facturacion = pd.read_excel(datos_modelo, sheet_name='fac_txn')
datos_producto = pd.read_excel(datos_modelo, sheet_name='product')
datos_clientes = pd.read_excel(datos_modelo, sheet_name='customer')
print('PRODUCTOS :'+ datos_producto.columns)
print('CLIENTES:'+ datos_clientes.columns)
print('FACTURACION:'+ datos_facturacion.columns)


In [0]:
datos_clientes.groupby('customer_id').size().value_counts()
# la mayoria de clientes solo tienen una compra

In [0]:
datos_facturacion.drop_duplicates('order_id',keep='first',inplace=True)

In [0]:
datos_facturacion['date'] = pd.to_datetime(datos_facturacion['order_purchase_timestamp']).dt.date
datos_facturacion = datos_facturacion.drop('order_purchase_timestamp',axis=1)
datos_facturacion.head()

In [0]:
transaction_data = datos_facturacion[['customer_id','date','total_value']]
transaction_data.head()

In [0]:
summary = summary_data_from_transaction_data(transaction_data,'customer_id','date',monetary_value_col='total_value',)
summary.describe()

In [0]:
summary.head()

In [0]:
summary[summary['frequency']>0].head()

In [0]:
datos_facturacion[datos_facturacion['customer_id']=='004288347e5e88a27ded2bb23747066c']

In [0]:
bgf = BetaGeoFitter(penalizer_coef=0.0)
bgf.fit(summary['frequency'], summary['recency'], summary['T']);

In [0]:
plot_frequency_recency_matrix(bgf);

In [0]:
plot_probability_alive_matrix(bgf);

In [0]:
plot_period_transactions(bgf).set_yscale('log');

In [0]:
summary_cal_holdout = calibration_and_holdout_data(datos_facturacion, 'customer_id', 'date',calibration_period_end='2017-09-03', observation_period_end='2018-09-03' )

In [0]:
bgf.fit(summary_cal_holdout['frequency_cal'], summary_cal_holdout['recency_cal'], summary_cal_holdout['T_cal'])
plot_cumulative_transactions(bgf, datos_facturacion, 'date', 'customer_id', 730, 365);

In [0]:
plot_incremental_transactions(bgf, datos_facturacion, 'date', 'customer_id', 730, 365);

In [0]:
plot_calibration_purchases_vs_holdout_purchases(bgf, summary_cal_holdout);

In [0]:
datos_clientes.groupby('customer_id').size().value_counts()

In [0]:
df = summary[summary['frequency']>0]
df['prob_alive'] = bgf.conditional_probability_alive(df['frequency'],df['recency'],df['T'])
sns.distplot(df['prob_alive']);

In [0]:
df['churn'] = ['churned' if p < .1 else 'not churned' for p in df['prob_alive']]
sns.countplot(df['churn']);

In [0]:
sns.distplot(df[df['churn']=='not churned']['prob_alive']).set_title('Probability alive, not churned');

In [0]:
df['churn'][(df['prob_alive']>=.1) & (df['prob_alive']<.2)] = "high risk"
df['churn'].value_counts()

In [0]:
#df[df['churn'] =='high risk'].head()
df[df['churn'] =='churned'].head()

In [0]:
df['definition'] = df['churn'].apply(lambda x: 1 if x == 'churned' else 0)

In [0]:
df['definition'].count()

In [0]:
datos_clientes.count()

In [0]:
prueba_clientes = pd.merge(datos_clientes,df[['definition']],on='customer_id', how='left')
prueba_clientes['definition'].fillna(0, inplace=True)
print(prueba_clientes.count())
prueba_clientes.head()

In [0]:
prueba_clientes['definition'].mean()

In [0]:

#df[1].fillna(0, inplace=True)